In [1]:
from pyspark.sql import SparkSession

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


In [2]:
spark = SparkSession \
    .builder \
    .appName('test') \
    .master('spark://spark-master:7077') \
    .config('spark.ui.port', '4051') \
    .config('spark.dynamicAllocation.enabled', 'true') \
    .config('spark.eventLog.enabled', 'true') \
    .config('spark.eventLog.dir', '/home/spark-events') \
    .config('spark.executor.cores', 1) \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/26 14:58:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
!mkdir -p /home/data/nyc_taxi_data/green_tripdata && \
    rm -f /home/data/nyc_taxi_data/green_tripdata/* && \
    wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet -P /home/data/nyc_taxi_data/green_tripdata/

--2023-10-26 14:58:38--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.225.242.58, 13.225.242.37, 13.225.242.202, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.225.242.58|:443... connected.


In [ ]:
df = spark.read.format('parquet').load('/home/data/nyc_taxi_data/green_tripdata/')

In [ ]:
df.count()

In [ ]:
df.printSchema()

In [ ]:
from pyspark.sql.functions import date_format

# fix error in pandas 2.1.1
# Casting to unit-less dtype 'datetime64' is not supported. Pass e.g. 'datetime64[ns]' instead.
df = df.withColumn("lpep_pickup_datetime", date_format("lpep_pickup_datetime", "yyyy-MM-dd HH:mm:ss"))
df = df.withColumn("lpep_dropoff_datetime", date_format("lpep_dropoff_datetime", "yyyy-MM-dd HH:mm:ss"))

In [ ]:
df.limit(10).toPandas()

In [ ]:
spark.stop()